In [31]:
using JuMP, Gurobi, CSV
pixels = CSV.read("output_ghana5_small.csv", header = 1)
model = Model(with_optimizer(Gurobi.Optimizer))
pixelsSize = size(pixels)   #tuple representing dimensions of country
xMax = pixelsSize[1]    
yMax = pixelsSize[2]
totalPop = sum(sum(pixels[i,j] for i in 1:xMax) for j in 1:yMax)
percentage = 0.95

@variable(model, S[1:xMax, 1:yMax], Bin)   #S[i,j] = 1 if there is a small access pt at (i,j)
@variable(model, L[1:xMax, 1:yMax], Bin)   #L[i,j] = 1 if there is a large access pt at (i,j)
TS = sum(sum(S[i,j] for j in 1:yMax) for i in 1:xMax)   #total number of small access pts
TL = sum(sum(L[i,j] for j in 1:yMax) for i in 1:xMax)   #total number of large access pts
@objective(model, Min, 150*TS+250TL)   #minimize total cost of access pts

R = zeros(pixelsSize[1], pixelsSize[2])
for i in 1:xMax        #loop through all points (i,j) in the country
    for j in 1:yMax
        @constraint(model, S[i,j]+L[i,j] <= 1)   #at most one access point per pixel
        if S[i,j] == 1    #if there is a small access point at (i,j)
            for x in max((i-5),1):min(xMax,(i+5))   #range of x values in range of wifi
                for y in max(-sqrt(25-x^2),1):min(yMax,sqrt(25-x^2))    #range of y values in range of wifi
                    R[x,y] = 1    #R[x,y] = 1 if (x,y) is within range of a small access point
                end
            end
        elseif L[i,j] == 1    #if there is a large access point at (i,j)
            for x in max((i-15),1):min(xMax,(i+15))   #range of x values in range of wifi
                for y in max(-sqrt(225-x^2),1):min(yMax,sqrt(225-x^2))    #range of y values in range of wifi
                    R[x,y] = 1    #R[x,y] = 1 if (x,y) is within range of a small access point
                end
            end
        end
    end
end

@constraint(model, percentage*totalPop<=sum(sum(pixels[i,j]*R[i,j] for i in 1:xMax) for j in 1:yMax))

optimize!(model)

println("cost: ", objective_value(model))

Academic license - for non-commercial use only

Consider calling update less frequently.

Optimize a model with 2081 rows, 4160 columns and 4160 nonzeros
Variable types: 0 continuous, 4160 integer (4160 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+07]
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


Gurobi.GurobiError: Gurobi.GurobiError(10005, "Unable to retrieve attribute 'ObjVal'")

In [ ]:
using JuMP, Gurobi, CSV
pixels = CSV.read("output_ghana_10km.csv", header = 1)
pixelsSize = size(pixels)   #tuple representing dimensions of country
xMax = pixelsSize[1]
yMax = pixelsSize[2]
totalPop = sum(sum(pixels[i,j] for i in 1:xMax) for j in 1:yMax)
percentage = 0.95
model = Model(with_optimizer(Gurobi.Optimizer))

@variable(model, S[1:xMax, 1:yMax], Bin)   # S[i,j] = 1 if there is a small access pt at (i,j)
@variable(model, L[1:xMax, 1:yMax], Bin)   # L[i,j] = 1 if there is a large access pt at (i,j)
@variable(model, R[1:xMax, 1:yMax], Bin)   # R[i,j] = 1 if (i,j) is covered by an access pt

TS = sum(sum(S[i,j] for j in 1:yMax) for i in 1:xMax)   # total number of small access pts
TL = sum(sum(L[i,j] for j in 1:yMax) for i in 1:xMax);  # total number of large access pts

@objective(model, Min, 75*TS+250*TL);   # minimize total cost of access pts

for i in 1:xMax      #loop through all points in the country
    for j in 1:yMax   #R[i,j] must be <= sum of all access points within range
        @constraint(model, R[i,j] <= S[i, j]   #small access points only cover size of one pixel
                                     +
                                     sum(sum(L[i+dx, j+dy]   #use distance formula to find range
                                     for dy in max(trunc(Int, -sqrt(1^2-dx^2)), 1-j) #min/max prevents out of range
                                            : min(yMax-j, trunc(Int, sqrt(1^2-dx^2))))
                                     for dx in max(-1, 1-i) : min(1, xMax-i)))
    end
end    

#at leat 95% of the population of the country must be covered
@constraint(model, sum(sum(pixels[i,j]*R[i,j] for i in 1:xMax) for j in 1:yMax) >= percentage*totalPop);

# Solving the optimization problem  
optimize!(model)